In [1]:
import numpy as np
import tensorflow as tf
# keras model approach
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dense, Input
from tensorflow.keras.activations import relu
from tqdm import tqdm
from collections import deque
import random
import gym 

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.set_floatx('float32')

In [3]:
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.render(mode="ipython", width=500, height=450)

Failed: football: No module named 'gfootball'


In [4]:
env = make("connectx", debug=True)
print(dir(env))
#print(env.state[0]['observation']['board'])
'''env.state[0]['observation']['board'][0] = 1
env.state[0]['observation']['board'][1] = 1
env.state[0]['observation']['board'][2] = 1
env.state[0]['observation']['board'][3] = 1'''
print(env.done)
env.step([1,1])
env.step([1,1])
a = env.step([1,1])
print("A")
print(a)
print(env.state[0]['observation']['board'])
env.render(mode="ipython", width=500, height=450)

['_Environment__agent_runner', '_Environment__get_shared_state', '_Environment__get_state', '_Environment__process_specification', '_Environment__run_interpreter', '_Environment__set_state', '_Environment__state_schema', '_Environment__state_schema_value', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__state_schema_0', '__state_schema_1', '__str__', '__subclasshook__', '__weakref__', 'agents', 'clone', 'configuration', 'debug', 'debug_print', 'done', 'html_renderer', 'id', 'info', 'interpreter', 'logs', 'name', 'play', 'pool', 'render', 'renderer', 'reset', 'run', 'specification', 'state', 'step', 'steps', 'toJSON', 'train', 'version']
False
A
[{'action': 1, 'reward': 0, 'info': {}, 'observation': {'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [5]:
action_space = env.configuration['columns']
obs = env.reset()
# creating a non CNN model with directly feeding all the squares to the NN
# The input has *3 because we want each piece to have a different class,
# so basically 3 matrices will be formed 1 for empty squares, 1 matrice for 
# player 1's coin, 1 matrice for player 2's coin 
# all will be concatenated and then fed into the NN

nn_input = len(obs[0]['observation']['board'])*3
print(nn_input)
env.render(mode="ipython", width=500, height=450)
# buffer size represents the last n turns (not moves) which means 
# it stores last 5 played moves irrespective of player 1 or 2
buffer_size = 6

126


In [6]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [7]:
def model_keras():
    
    inputs = Input(shape=(nn_input*buffer_size,))
    x = Dense(1000,activation='linear',kernel_initializer="glorot_uniform")(inputs)
    x = BatchNormalization()(x)
    output = Dense(action_space,activation='linear',kernel_initializer="glorot_uniform")(x)
    model = Model(inputs=inputs, outputs=output, name="RL_Value_Function")
    
    print(model.summary())
    
    model.compile(optimizer=adam,loss='mean_squared_error',metrics=['mean_squared_error'])
    
    return model
# converts observation into NN input type
def encode(board):
    empty = []
    player_1 = []
    player_2 = []
    
    for i in board:
        if i == 0:
            empty.append(1)
        else:
            empty.append(0)
        if i == 1:
            player_1.append(1)
        else:
            player_1.append(0)
        if i == 2:
            player_2.append(2)
        else:
            player_2.append(0)
            
    output = np.concatenate((np.array(empty),np.array(player_1),np.array(player_2)),axis=0)
    return output
model_1 = model_keras()
model_2 = model_keras()

Model: "RL_Value_Function"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 756)]             0         
_________________________________________________________________
dense (Dense)                (None, 1000)              757000    
_________________________________________________________________
batch_normalization (BatchNo (None, 1000)              4000      
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 7007      
Total params: 768,007
Trainable params: 766,007
Non-trainable params: 2,000
_________________________________________________________________
None
Model: "RL_Value_Function"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 756)]             0         
_______

In [8]:
'''class ConnectX(gym.Env):
    def __init__(self, switch_prob=0.0):
        self.env = make('connectx', debug=False)
        self.pair = [None, 'random']
        self.trainer = self.env.train(self.pair)
        self.switch_prob = switch_prob

        # Define required gym fields (examples):
        config = self.env.configuration
        self.action_space = gym.spaces.Discrete(config.columns)
        self.observation_space = gym.spaces.Discrete(config.columns * config.rows)

    def switch_trainer(self):
        self.pair = self.pair[::-1]
        self.trainer = self.env.train(self.pair)

    def step(self, action):
        return self.trainer.step(action)
    
    def reset(self):
        if np.random.random() < self.switch_prob:
            self.switch_trainer()
        return self.trainer.reset()
    
    def render(self, **kwargs):
        return self.env.render(**kwargs)'''

"class ConnectX(gym.Env):\n    def __init__(self, switch_prob=0.0):\n        self.env = make('connectx', debug=False)\n        self.pair = [None, 'random']\n        self.trainer = self.env.train(self.pair)\n        self.switch_prob = switch_prob\n\n        # Define required gym fields (examples):\n        config = self.env.configuration\n        self.action_space = gym.spaces.Discrete(config.columns)\n        self.observation_space = gym.spaces.Discrete(config.columns * config.rows)\n\n    def switch_trainer(self):\n        self.pair = self.pair[::-1]\n        self.trainer = self.env.train(self.pair)\n\n    def step(self, action):\n        return self.trainer.step(action)\n    \n    def reset(self):\n        if np.random.random() < self.switch_prob:\n            self.switch_trainer()\n        return self.trainer.reset()\n    \n    def render(self, **kwargs):\n        return self.env.render(**kwargs)"

In [9]:
def custom_loss(y_true,y_pred):
    return tf.keras.losses.mean_squared_error(y_true,y_pred)

In [10]:
import random
random.seed(2020)

#@tf.function
def batch_train(model_1,model_2,gamma,batch_size,epsilon):
    
    #decaying the exploration
    if epsilon['epsilon'] > epsilon['epsilon_min']:
         epsilon['epsilon'] =  epsilon['epsilon'] * epsilon['epsilon_decay']
    batch = random.sample(replay_batch,batch_size)
    
    batch_reward = []
    batch_action = []
    batch_done = []
    
    batch_current_state = np.zeros((batch_size, nn_input*buffer_size))
    batch_next_state = np.zeros((batch_size, nn_input*buffer_size))

    for i in range(batch_size):
        batch_reward.append(batch[i][2])
        batch_action.append(batch[i][1])
        batch_current_state[i] = batch[i][0]
        batch_next_state[i] = batch[i][3]
        batch_done.append(batch[i][4])
    #lets calculate the next state value as the current value will be calculated in 
    # in gradient tape
    
    next_q = model_2.predict(batch_next_state)
    
    max_q = []
    for i in next_q:
        max_q.append(max(i))
    max_q = np.array(max_q,dtype = 'float32')
    
    target = batch_reward + gamma*max_q # this is the Q learning Target
    
    
    with tf.GradientTape() as tape:
        # logits is the forward pass
        logits = model_1(batch_current_state, training=True)
        
        q_target = np.array(logits)
        
        
        # VERY IMPORTANT NOTE, IF THE EPISODE ENDS THE DONE VALUE BECOMES TRUE
        # IT IS VERY IMPORTANT THAT THE NN UPDATES TOWARDS THIS TRUE VALUE RATHER THAN
        # ITS OWN THINKING VALUE (r + gamma*max(action)) THAT WE USE FOR ALL
        # NON TERMINAL REWARDS 
        # THIS MAKES OR BREAKS THE NETWORK VERY VERY IMPORTANT
        for i in range(batch_size):
            q_target[i][batch_action[i]] = target[i]
            if batch_done[i]:
                q_target[i][batch_action[i]] = batch_reward[i]

        # calculating the loss
        loss_value = custom_loss(q_target,logits)
    
    #we retrieve the gradients
    grads = tape.gradient(loss_value, model_1.trainable_weights)
    
    #THIS IS ONE STEP OF GRAD DESCENT (Minimizes the loss)
    adam.apply_gradients(zip(grads, model_1.trainable_weights))

def policy(q_vals,turn):
    # lets implement a policy which decays
    if np.random.rand() <= epsilon['epsilon']:  
        return random.randrange(action_space)
    elif turn == True:
        action = np.argmax(q_vals[0])
        return action
    else:
        action = np.argmin(q_vals[0])
        return action
def update_target_network():
    model_2.set_weights(model_1.get_weights())
# this is the custom reward function
def get_reward(rew):
    reward = None
    if rew == 1:
        reward = 1
    elif rew == -1:
        reward = -1
    elif rew == None:
        reward = -5
    else:
        reward = 0
    return reward


In [11]:
env.reset()

x = env.step([0,1])
x = env.step([0,1])
x = env.step([0,1])
x = env.step([0,1])
x = env.step([0,1])
x = env.step([0,1])
x = env.step([0,1])
print(env.done)
print(x)
print(encode(x[0]['observation']['board']))
env.render(mode="ipython", width=300, height=300)

True
[{'action': 0, 'reward': 1, 'info': {}, 'observation': {'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0], 'mark': 1}, 'status': 'DONE'}, {'action': 1, 'reward': -1, 'info': {}, 'observation': {'mark': 2}, 'status': 'DONE'}]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0
 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0]


In [12]:
# lets build some memory into the model to perform decorrelated batch updates
# this is TD learning

# so apparantly the NN has to be adjusted only according to the action taken by it
# for example if action 2 is taken then only the weights for action 2 should be changed
# for this reason we should only update the q_2 vector with regarding the chosen action
# rewards will also be added to that action only
# q_2 is what we thought the value of the state will be after doing action
# we also add the reward and make this the target for the NN

replay_batch = deque(maxlen = 3000)

warmup = 10 #will start training after these many episodes have passed

# to balance exploration
epsilon = {
"epsilon" : 1.0,
"epsilon_decay": 0.999,
"epsilon_min":0.01,
}

In [13]:
# THE TRAINING LOOP
global_steps = 0
# to have same networks in the starting
update_target_network()

for i in tqdm(range(10000)):
    observation = env.reset()[0]['observation']['board']
    observation = encode(observation)
    done = False
    turn = True
    total_reward = 0
    current_buffer = deque(maxlen = buffer_size)
    next_buffer = deque(maxlen = buffer_size)
    for j in range(buffer_size):
        current_buffer.append(np.full(nn_input,-1))
        next_buffer.append(np.full(nn_input,-1))

    next_buffer.append(observation)
    while not done:        
        #storing the current state
        state_1 = observation
        current_buffer.append(observation)
        input_network = np.expand_dims(np.array(current_buffer).flatten(), axis=0)
        # this is the current q values
        q_state = model_1(input_network,training = False)
        
        action = policy(q_state,turn) # current action
        
        # ok so this environments works a bit different 
        # actions of player 1 have to be in the format ['player 1 action',0]
        # actions of player 2 havet to be in the format [0,'player 2']
        st = None
        if turn == True:
            st = env.step([int(action),0])
            turn = False
        else:
            st = env.step([0,int(action)])
            turn = True
        
        next_state = st[0]['observation']['board']
        next_state = encode(next_state)
        reward = get_reward(st[0]['reward'])
        done = env.done
        
        # calculating the total reward
        total_reward = total_reward + reward
        state_reward = reward
        
        observation = next_state
        
        next_buffer.append(observation)
        next_input_network = np.expand_dims(np.array(next_buffer).flatten(), axis=0)
        # only store for player 1
        if turn == True:
            replay_batch.append((input_network,action,state_reward,next_input_network,done))
        
        if i>warmup:
            batch_train(model_1,model_2,0.99,64,epsilon)
            global_steps = global_steps + 1
            
        if done:
            update_target_network()
            
    print(total_reward)

  0%|          | 2/10000 [00:00<52:22,  3.18it/s]  

0
-1


  0%|          | 4/10000 [00:00<41:11,  4.04it/s]

-1
-1


  0%|          | 5/10000 [00:00<34:44,  4.80it/s]

...

 42%|████▏     | 4190/10000 [42:17<3:27:39,  2.14s/it]

1


Exception ignored in: <bound method IteratorResourceDeleter.__del__ of <tensorflow.python.data.ops.iterator_ops.IteratorResourceDeleter object at 0x000001D19EB6D7B8>>
Traceback (most recent call last):
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow_core\python\data\ops\iterator_ops.py", line 537, in __del__
    handle=self._handle, deleter=self._deleter)
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow_core\python\ops\gen_dataset_ops.py", line 1141, in delete_iterator
    deleter)
KeyboardInterrupt: 
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-929559e55f64>", line 58, in <module>
    batch_train(model_1,model_2,0.99,64,epsilon)
  File "<ipython-input-10-d69eed2ed919>", line 28, in batch_train
    next_q = model_2.predict(batch_next_state)
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 909, in predict
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 462, in predict
    steps=steps, callbacks=callbacks, **kwargs)
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 444, in _model_iteration
    total_epochs=1)
  File "C:\


KeyboardInterrupt



In [ ]:
model_1.save("G:\Data Science\Reinforcement Learning\Connect X kaggle/model_1_1000ep_DQN_n_turn.h5")
model_2.save("G:\Data Science\Reinforcement Learning\Connect X kaggle/model_2_1000ep_DQN_n_turn.h5")

In [14]:
# JUST TESTING THE NETWORK

for i in tqdm(range(50)):
    observation = env.reset()[0]['observation']['board']
    observation = encode(observation)
    done = False
    turn = True
    total_reward = 0
    current_buffer = deque(maxlen = buffer_size)
    next_buffer = deque(maxlen = buffer_size)
    for j in range(buffer_size):
        current_buffer.append(np.full(nn_input,-1))
        next_buffer.append(np.full(nn_input,-1))

    next_buffer.append(observation)
    while not done:        
        #storing the current state
        state_1 = observation
        current_buffer.append(observation)
        input_network = np.expand_dims(np.array(current_buffer).flatten(), axis=0)
        # this is the current q values
        q_state = model_1(input_network,training = False)
        
        action = policy(q_state,turn) # current action
        
        # ok so this environments works a bit different 
        # actions of player 1 have to be in the format ['player 1 action',0]
        # actions of player 2 havet to be in the format [0,'player 2']
        st = None
        if turn == True:
            st = env.step([int(action),0])
            turn = False
        else:
            st = env.step([0,int(action)])
            turn = True
        
        next_state = st[0]['observation']['board']
        next_state = encode(next_state)
        reward = get_reward(st[0]['reward'])
        done = env.done
        
        # calculating the total reward
        total_reward = total_reward + reward
        state_reward = reward
        
        observation = next_state
        
        next_buffer.append(observation)
        next_input_network = np.expand_dims(np.array(next_buffer).flatten(), axis=0)
    env.render(mode="ipython", width=300, height=300)
    print(total_reward)


  0%|          | 0/50 [00:00<?, ?it/s]


  2%|▏         | 1/50 [00:00<00:08,  6.02it/s]

1



  4%|▍         | 2/50 [00:00<00:09,  5.24it/s]

1


...


 32%|███▏      | 16/50 [00:03<00:07,  4.44it/s]

1



 34%|███▍      | 17/50 [00:03<00:07,  4.41it/s]

1


KeyboardInterrupt: 

In [ ]:
epsilon

In [24]:
import json
from os import path
from random import choice

EMPTY = 0


def play(board, column, mark, config):
    columns = config.columns
    rows = config.rows
    row = max([r for r in range(rows) if board[column + (r * columns)] == EMPTY])
    board[column + (row * columns)] = mark


def is_win(board, column, mark, config, has_played=True):
    columns = config.columns
    rows = config.rows
    inarow = config.inarow - 1
    row = (
        min([r for r in range(rows) if board[column + (r * columns)] == mark])
        if has_played
        else max([r for r in range(rows) if board[column + (r * columns)] == EMPTY])
    )

    def count(offset_row, offset_column):
        for i in range(1, inarow + 1):
            r = row + offset_row * i
            c = column + offset_column * i
            if (
                r < 0
                or r >= rows
                or c < 0
                or c >= columns
                or board[c + (r * columns)] != mark
            ):
                return i - 1
        return inarow

    return (
        count(1, 0) >= inarow  # vertical.
        or (count(0, 1) + count(0, -1)) >= inarow  # horizontal.
        or (count(-1, -1) + count(1, 1)) >= inarow  # top left diagonal.
        or (count(-1, 1) + count(1, -1)) >= inarow  # top right diagonal.
    )


def random_agent(obs, config):
    return choice([c for c in range(config.columns) if obs.board[c] == EMPTY])


def negamax_agent(obs, config):
    columns = config.columns
    rows = config.rows
    size = rows * columns

    # Due to compute/time constraints the tree depth must be limited.
    max_depth = 4

    def negamax(board, mark, depth):
        moves = sum(1 if cell != EMPTY else 0 for cell in board)

        # Tie Game
        if moves == size:
            return (0, None)

        # Can win next.
        for column in range(columns):
            if board[column] == EMPTY and is_win(board, column, mark, config, False):
                return ((size + 1 - moves) / 2, column)

        # Recursively check all columns.
        best_score = -size
        best_column = None
        for column in range(columns):
            if board[column] == EMPTY:
                # Max depth reached. Score based on cell proximity for a clustering effect.
                if depth <= 0:
                    row = max(
                        [
                            r
                            for r in range(rows)
                            if board[column + (r * columns)] == EMPTY
                        ]
                    )
                    score = (size + 1 - moves) / 2
                    if column > 0 and board[row * columns + column - 1] == mark:
                        score += 1
                    if (
                        column < columns - 1
                        and board[row * columns + column + 1] == mark
                    ):
                        score += 1
                    if row > 0 and board[(row - 1) * columns + column] == mark:
                        score += 1
                    if row < rows - 2 and board[(row + 1) * columns + column] == mark:
                        score += 1
                else:
                    next_board = board[:]
                    play(next_board, column, mark, config)
                    (score, _) = negamax(next_board,
                                         1 if mark == 2 else 2, depth - 1)
                    score = score * -1
                if score > best_score or (score == best_score and choice([True, False])):
                    best_score = score
                    best_column = column

        return (best_score, best_column)

    _, column = negamax(obs.board[:], obs.mark, max_depth)
    if column == None:
        column = choice([c for c in range(columns) if obs.board[c] == EMPTY])
    return column

In [25]:
env.play([None, negamax_agent], width=500, height=450)